# C2SIM

In [9]:
from jax import numpy as jit, vmap, random
from jaxmarl import make

from einops import rearrange
from functional import partial
from tqdm import tqdm
import yaml

from src import plot_fn, make_bt, traj_fn

In [10]:
# globals
model = "mistral:7b-instruct-q2_K"
bt_path = "data/bt.yaml"
scenario = "3s5z_vs_3s6z"

In [11]:
with open("config.yaml") as f:
    config = yaml.load(f, Loader=yaml.FullLoader)
    n_envs = config["n_envs"]
    n_steps = config["n_steps"]
    n_allies = config["n_allies"]
    n_enemies = config["n_enemies"]

## language model

## smax trajectories

In [12]:
# scenario=map_name_to_scenario(scenario))  # TODO: allow HeuristicSMAX
env = make("SMAX", num_allies=n_allies, num_enemies=n_enemies)
bt = jit(vmap(make_bt(env, bt_path), in_axes=(0, 0, None)))
rng, key = random.split(random.PRNGKey(0))
state_seq, reward_seq = traj_fn(bt, key, env)
# plot_fn(env, state_seq, reward_seq, expand=True)

TypeError: 'module' object is not callable

## language model